In [11]:
# Step 1: Retrieve data from the API endpoint
import requests
import json
import pandas as pd
# !pip install --upgrade pandas


from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv('.env')

# Access the values
qux_auth_token = os.getenv('qux_auth')
qux_cookie = os.getenv('qux_cookie')
qux_app = os.getenv('qux_app')
# headers as provided
headers = {
    'accept': 'application/json',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-GB,en;q=0.5',
    'authorization': qux_auth_token,  # replace <your-token> with actual token
    'content-type': 'application/json',
    'cookie': qux_cookie
}

In [12]:
# generate url based on app id
url = "https://quant-ux.com/rest/apps/" + qux_app + ".json?exclude=Animation"
response = requests.get(url, headers=headers)

# Step 2: Parse the JSON into a Python object
data = json.loads(response.text)

# Convert to DataFrame for easier analysis
df_app = pd.DataFrame(data)
df_app_screens = df_app[df_app['screens'].notna()]
df_app_widgets = df_app[df_app['widgets'].notna()]

# create a list of all screens with id and name
screen_list = []
for screen in df_app_screens['screens']:
    screen_list.append({'id': screen['id'], 'name': screen['name']})

widget_list = []

for widget in df_app_widgets['widgets']:
    widget_list.append({'id': widget['id'], 'name': widget['name']})

# create a list of all elements with id, name, type and screen_id
element_list = []

# print all screens


def print_screens():
    for screen in screen_list:
        print(screen['name'])   

# print all widgets

print_screens()


WelcomeScreen - InfrastructureGPT
SurveyScreen
SQ1_Parking
SQ2_UnsafeIntersection
SQ3_DarkBikePath
SQ4_DangerousLitter
SQ5_BikeToSchool
SQ6_ServiceStations
SQ7_SlowTrafficLights
Consent
Project Intro
ThankYouScreen
Demographics
Feedback


In [13]:

url = "https://quant-ux.com/rest/events/" + qux_app + ".json?exclude=Animation"

response = requests.get(url, headers=headers)

# Step 2: Parse the JSON into a Python object
data = json.loads(response.text)

# Convert to DataFrame for easier analysis
df = pd.DataFrame(data)
# df['state'] = pd.json_normalize(df['state'])
# df['state'] = pd.json_normalize(df['state'])

state_df = pd.json_normalize(df['state'])
state_df = state_df[['type', 'value']]

# Step 3: Perform your analysis
# (This step depends on what exactly you want to do with the data)
# Here, we'll just show the first few rows of the DataFrame
print(df.head())
# print(state_df.head())

                        _id              session   
0  6467a86d7001be11c15e906e  S1684514774496_4526  \
1  6467a8757001be11c15e907e  S1684514774496_4526   
2  646b4fe57001be11c15f5396   S1684754314198_343   
3  6467a7d67001be11c15e900e  S1684514774496_4526   
4  6467a7dc7001be11c15e903d  S1684514774496_4526   

                                              user        screen widget   
0  {'id': 'U1684514774496_5907', 'name': 'tester'}   s10212_1967   None  \
1  {'id': 'U1684514774496_5907', 'name': 'tester'}   s10212_1967   None   
2  {'id': 'U1684754314198_4209', 'name': 'tester'}  s10047_72116   None   
3  {'id': 'U1684514774496_5907', 'name': 'tester'}  s10004_18897   None   
4  {'id': 'U1684514774496_5907', 'name': 'tester'}  s10178_14944   None   

           type           time  scrollTop      x      y   
0   ScreenClick  1684514925350        0.0  0.677  0.213  \
1   ScreenClick  1684514933609        0.0  0.139  0.435   
2   ScreenClick  1684754405641        0.0  0.858  0.873   


In [14]:
# get screen and widget names from screen_names and widget_names using the id

df['screen_name'] = df['screen'].apply(
    lambda x: next((item for item in screen_list if item["id"] == x), {'name': None})['name']
)

df['widget_name'] = df['widget'].apply(
    lambda x: next((item for item in widget_list if item["id"] == x), {'name': None})['name']
)


In [15]:
# df[df['type'] == ¿']
df['stype'] = df['state'].apply(lambda x: x.get('type') if isinstance(x, dict) else None)
df['svalue'] = df['state'].apply(lambda x: x.get('value') if isinstance(x, dict) else None)


In [16]:
# filter the df to only show the relevant events meaning they have a value and a widget_name

df = df[df['svalue'].notna()] # filter out all events without a value
df = df[df['widget_name'].notna()] # filter out all events without a widget_name


In [17]:
# output the df columns

print(df.columns)

Index(['_id', 'session', 'user', 'screen', 'widget', 'type', 'time',
       'scrollTop', 'x', 'y', 'appID', 'id', 'device', 'state', 'noheat',
       'screen_name', 'widget_name', 'stype', 'svalue'],
      dtype='object')


In [18]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns





In [19]:
# Print first 5 rows of data
print(df.head())

# Print descriptive statistics
print(df.describe())

# Print data types of each column
print(df.dtypes)

# For categorical data, look at frequency distribution
# print(df['stype'].value_counts())
# print(df['widget_name'].value_counts())
# print(df['screen_name'].value_counts())


# Create a new column 'svalue_num' that converts 'svalue' to numeric, non-numeric becomes NaN
df['svalue_num'] = pd.to_numeric(df['svalue'], errors='coerce')

# Display the DataFrame to verify the operation
print(df.head())
# Compute the average value for each widget_name
average_values_per_widget = df.groupby('widget_name')['svalue_num'].mean()
print(average_values_per_widget)
# Compute the average value for widget names containing "Creativity", "Effectiveness", and "Feasibility"
average_values_creativity = df[df['widget_name'].str.contains('Creativity')]['svalue_num'].mean()
average_values_effectiveness = df[df['widget_name'].str.contains('Effectiveness')]['svalue_num'].mean()
average_values_feasibility = df[df['widget_name'].str.contains('Feasibility')]['svalue_num'].mean()

# Create a DataFrame from the computed averages
overview_df = pd.DataFrame({
    'Widget Category': ['Creativity', 'Effectiveness', 'Feasibility'],
    'Average Value': [average_values_creativity, average_values_effectiveness, average_values_feasibility]
})

# Display the overview DataFrame
print(overview_df)


                         _id              session   
43  6467a8757001be11c15e907d  S1684514774496_4526  \
44  6467a8767001be11c15e9082  S1684514774496_4526   
45  6467a8777001be11c15e9084  S1684514774496_4526   
46  6467a8777001be11c15e9085  S1684514774496_4526   
47  6467a8787001be11c15e9087  S1684514774496_4526   

                                               user       screen   
43  {'id': 'U1684514774496_5907', 'name': 'tester'}  s10212_1967  \
44  {'id': 'U1684514774496_5907', 'name': 'tester'}  s10212_1967   
45  {'id': 'U1684514774496_5907', 'name': 'tester'}  s10212_1967   
46  {'id': 'U1684514774496_5907', 'name': 'tester'}  s10212_1967   
47  {'id': 'U1684514774496_5907', 'name': 'tester'}  s10212_1967   

          widget          type           time  scrollTop    x    y   
43  w10233_71694  WidgetChange  1684514933609        0.0 -1.0 -1.0  \
44  w10231_20688  WidgetChange  1684514934531        0.0 -1.0 -1.0   
45  w10231_20688  WidgetChange  1684514935025        0.0 -1.0 